In [1]:
import pymysql
import pandas as pd

try:
    # Conexión a la base de datos del simulador
    connection = pymysql.connect(
        host='212.227.90.6',
        user='Equipo12',
        password='E1q2u3i4p5o12',
        database='Equip_12',
        port=3306
    )
    print("✅ Conexión establecida con MySQL Server")

    # Crear un cursor
    cursor = connection.cursor()

    # Consultar las tablas disponibles
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()

    # Diccionario para almacenar los dataframes
    dataframes = {}

    for tabla in tables:
        table_name = tabla[0]
        print(f"📥 Cargando tabla: {table_name}")
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql(query, connection)
        dataframes[table_name] = df
        print(f"✅ DataFrame para '{table_name}' creado con {df.shape[0]} filas")

except pymysql.MySQLError as e:
    print("❌ Error al conectar con MySQL:", e)


✅ Conexión establecida con MySQL Server
📥 Cargando tabla: RRHH
✅ DataFrame para 'RRHH' creado con 740 filas


C:\Users\dessb\AppData\Local\Temp\ipykernel_8512\2277306723.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [2]:
df_rrhh = dataframes['RRHH']

In [3]:
df_rrhh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       740 non-null    int64 
 1   Reason_absence           740 non-null    int64 
 2   Month_absence            740 non-null    int64 
 3   Day_week                 740 non-null    int64 
 4   Seasons                  740 non-null    int64 
 5   Transportation_expense   740 non-null    int64 
 6   Distance_Residence_Work  740 non-null    int64 
 7   Service_time             740 non-null    int64 
 8   Age                      740 non-null    int64 
 9   Work_load_Average_day    740 non-null    object
 10  Hit_target               740 non-null    int64 
 11  Disciplinary_failure     740 non-null    object
 12  Education                740 non-null    object
 13  Son                      740 non-null    object
 14  Social_drinker           740 non-null    o

1- Tratamiento de valores faltantes

In [4]:
df_rrhh.isna().sum()

# No hay nulos

ID                         0
Reason_absence             0
Month_absence              0
Day_week                   0
Seasons                    0
Transportation_expense     0
Distance_Residence_Work    0
Service_time               0
Age                        0
Work_load_Average_day      0
Hit_target                 0
Disciplinary_failure       0
Education                  0
Son                        0
Social_drinker             0
Social_smoker              0
Pet                        0
Weight                     0
Height                     0
Body_mass_index            0
Absenteeism_hours          0
dtype: int64

2- Eliminación o corrección de duplicados

In [5]:
# Duplicados completos
df_rrhh.duplicated().sum()

np.int64(34)

In [6]:
# Ver los duplicados concretos
df_rrhh[df_rrhh.duplicated()]

# Se han detectado 34 filas completamente duplicadas, algunas repetidas hasta 7 veces, como en el caso del ID 3 con el mismo motivo, mes y carga horaria.
# Aunque un mismo empleado puede tener varias ausencias similares, la duplicación exacta de todos los campos en múltiples filas consecutivas sugiere errores de carga de datos.
# Recomiendo dejarlo marcado como “candidato a eliminación” y decidirlo con el equipo antes de aplicar drop_duplicates().


,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Work_load_Average_day,...,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_hours
92,10,22,12,4,4,361,52,3,28,"261,306",...,0,1,1,1,0,4,80,172,27,8
348,34,23,10,3,4,118,10,10,37,"253,465",...,0,1,0,0,0,0,83,172,28,3
383,3,27,2,4,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
386,3,27,2,4,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
387,3,27,2,6,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
388,3,27,2,4,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
389,3,27,2,6,2,179,51,18,38,"251,818",...,0,1,0,1,0,0,89,170,31,3
396,22,23,5,4,3,179,26,9,30,"246,074",...,0,3,0,0,0,0,56,171,19,3
412,28,23,12,4,4,225,26,9,28,"280,549",...,0,1,1,0,0,2,69,169,24,3
431,3,27,3,5,2,179,51,18,38,"222,196",...,0,1,0,1,0,0,89,170,31,3


3- Validación de valores atípicos (outliers)

In [7]:
df_rrhh.describe()

# Se han encontrado posibles valores extremos en:
# Transportation_expense (máx. 388), la media es 221
# Absenteeism_hours (máx. 120)
# Work_load_Average_day (máx. 310), muy cercano al máximo permitido en carga diaria

,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Hit_target,Weight,Height,Body_mass_index,Absenteeism_hours
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,18.017568,19.216216,6.324324,3.914865,2.544595,221.329730,29.631081,12.554054,36.450000,94.587838,79.035135,172.114865,26.677027,6.924324
std,11.021247,8.433406,3.436287,1.421675,1.111831,66.952223,14.836788,4.384873,6.478772,3.779313,12.883211,6.034995,4.285452,13.330998
min,1.000000,0.000000,0.000000,2.000000,1.000000,118.000000,5.000000,1.000000,27.000000,81.000000,56.000000,163.000000,19.000000,0.000000
25%,9.000000,13.000000,3.000000,3.000000,2.000000,179.000000,16.000000,9.000000,31.000000,93.000000,69.000000,169.000000,24.000000,2.000000
50%,18.000000,23.000000,6.000000,4.000000,3.000000,225.000000,26.000000,13.000000,37.000000,95.000000,83.000000,170.000000,25.000000,3.000000
75%,28.000000,26.000000,9.000000,5.000000,4.000000,260.000000,50.000000,16.000000,40.000000,97.000000,89.000000,172.000000,31.000000,8.000000
max,36.000000,28.000000,12.000000,6.000000,4.000000,388.000000,52.000000,29.000000,58.000000,100.000000,108.000000,196.000000,38.000000,120.000000


4- Estandarización de formatos

In [8]:
df_rrhh['Month_absence'].value_counts()

# En la columna Month_absence se ha detectado el valor 0, que no corresponde a ningún mes válido (1–12). Solo hay 3 registros con este valor. Se sugiere marcarlo como anómalo y decidir si imputarlo (por moda o mediana), dejarlo como está o excluirlo del análisis final.


Month_absence
3     87
2     72
10    71
7     67
5     64
11    63
6     54
8     54
4     53
9     53
1     50
12    49
0      3
Name: count, dtype: int64

In [9]:
df_rrhh['Disciplinary_failure'].value_counts()

Disciplinary_failure
0    700
1     40
Name: count, dtype: int64

In [10]:
df_rrhh['Education'].value_counts()

Education
1    611
3     79
2     46
4      4
Name: count, dtype: int64

In [11]:
df_rrhh['Social_drinker'].value_counts()

Social_drinker
1    420
0    320
Name: count, dtype: int64

In [12]:
df_rrhh['Social_smoker'].value_counts()

Social_smoker
0    686
1     54
Name: count, dtype: int64

5- Corrección de tipos de datos

In [13]:
df_rrhh.dtypes
#En la revisión de tipos de datos se ha detectado que 7 columnas aparecen como object cuando no deberían:
# Work_load_Average_day debería ser float, pero contiene comas como separadores
# Disciplinary_failure, Social_drinker, Social_smoker, Education, Son y Pet deberían ser int o category

ID                          int64
Reason_absence              int64
Month_absence               int64
Day_week                    int64
Seasons                     int64
Transportation_expense      int64
Distance_Residence_Work     int64
Service_time                int64
Age                         int64
Work_load_Average_day      object
Hit_target                  int64
Disciplinary_failure       object
Education                  object
Son                        object
Social_drinker             object
Social_smoker              object
Pet                        object
Weight                      int64
Height                      int64
Body_mass_index             int64
Absenteeism_hours           int64
dtype: object

In [14]:
pd.set_option('display.max_columns', None) # para ver todas las columnas y que no se corten por el limite de Panda


In [15]:
df_rrhh.sample(40)

,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Work_load_Average_day,Hit_target,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_hours
481,34,27,1,4,2,118,10,10,37,"308,593",95,0,1,0,0,0,0,83,172,28,2
295,27,7,5,4,3,184,42,7,27,"378,884",92,0,1,0,0,0,0,58,167,21,4
345,34,23,10,3,4,118,10,10,37,"253,465",93,0,1,0,0,0,0,83,172,28,3
332,33,14,11,5,4,248,25,14,47,"268,519",93,0,1,2,0,0,1,86,165,32,4
570,3,27,2,5,2,179,51,18,38,"264,249",97,0,1,0,1,0,0,89,170,31,2
138,10,22,7,3,1,361,52,3,28,"275,312",98,0,1,1,1,0,4,80,172,27,8
55,30,11,2,4,2,157,27,6,29,"251,818",96,0,1,0,1,1,0,75,185,22,16
5,11,19,3,2,3,289,36,13,33,"222,196",99,0,1,2,1,0,1,90,172,30,104
417,3,27,1,5,2,179,51,18,38,"313,532",96,0,1,0,1,0,0,89,170,31,3
154,11,22,10,6,4,289,36,13,33,"265,017",88,0,1,2,1,0,1,90,172,30,8


# Nuevo DataFrame con una fila por empleado

In [16]:
# Paso 1: Ordenamos por ID y Age en orden descendente (mayor edad = más reciente)
df_rrhh_ordenado = df_rrhh.sort_values(by=['ID', 'Age'], ascending=[True, False]).copy()

df_rrhh_ordenado

,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Work_load_Average_day,Hit_target,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_hours
50,1,13,6,6,3,235,11,14,37,"377,550",94,0,3,1,0,0,1,88,172,29,16
64,1,22,7,2,1,235,11,14,37,"239,554",97,0,3,1,0,0,1,88,172,29,8
89,1,26,12,4,4,235,11,14,37,"261,306",97,0,3,1,0,0,1,88,172,29,8
145,1,19,8,5,1,235,11,14,37,"265,615",94,0,3,1,0,0,1,88,172,29,8
178,1,22,3,2,2,235,11,14,37,"244,387",98,0,3,1,0,0,1,88,172,29,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,36,0,9,3,4,118,13,18,50,"241,476",92,1,1,1,1,0,0,98,178,31,0
702,36,0,10,4,4,118,13,18,50,"253,465",93,1,1,1,1,0,0,98,178,31,0
714,36,0,10,6,4,118,13,18,50,"265,017",88,1,1,1,1,0,0,98,178,31,0
727,36,0,3,5,3,118,13,18,50,"244,387",98,1,1,1,1,0,0,98,178,31,0


In [17]:
# Paso 2: Tomamos únicamente el registro con la edad más alta por empleado
df_empleados = df_rrhh_ordenado.groupby('ID').first().reset_index()

df_empleados

,ID,Reason_absence,Month_absence,Day_week,Seasons,Transportation_expense,Distance_Residence_Work,Service_time,Age,Work_load_Average_day,Hit_target,Disciplinary_failure,Education,Son,Social_drinker,Social_smoker,Pet,Weight,Height,Body_mass_index,Absenteeism_hours
0,1,13,6,6,3,235,11,14,37,"377,550",94,0,3,1,0,0,1,88,172,29,16
1,2,18,8,5,1,235,29,12,48,"205,917",92,0,1,1,0,1,5,88,163,33,8
2,3,13,12,3,4,179,51,18,38,"280,549",98,0,1,0,1,0,0,89,170,31,32
3,4,0,0,3,1,118,14,13,40,"271,219",95,0,1,1,1,0,8,98,170,34,0
4,5,13,5,2,3,235,20,13,43,"246,074",99,0,1,1,1,0,0,106,167,38,16
5,6,22,7,3,1,189,29,13,33,"264,604",93,0,1,2,0,0,2,69,167,25,16
6,7,14,3,5,3,279,5,14,39,"343,253",95,0,1,2,1,1,0,68,168,24,16
7,8,0,9,3,1,231,35,14,39,"294,217",81,1,1,2,1,0,2,100,170,35,0
8,9,6,7,3,1,228,14,16,58,"264,604",93,0,1,2,0,0,1,65,172,22,120
9,10,13,8,2,1,361,52,3,28,"205,917",92,0,1,1,1,0,4,80,172,27,40


### Problema 1:
Un empleado puede tener varios registros con la misma edad máxima.

Algunas variables importantes, como Work_load_Average_day y Hit_target, fluctúan en los registros con la misma edad máxima.

### Propuesta 1:
Para la variables fluctuantes (workload y hit target) hacer el promedio, pero solo para los registros donde Age == edad máxima

### Problema 2: 
Algunas variables categóricas como Reason_absence, Month_absence, Day_week o Seasons se repiten múltiples veces para cada empleado, pero no son constantes.

### Propuesta 2:

Calcular la moda (valor más frecuente) de cada una de estas variables, pero solo considerando los registros donde el empleado tenía su edad más alta (es decir, su etapa más reciente en la empresa).  

Esto nos permitirá capturar un patrón de comportamiento típico de cada empleado en su etapa actual.

### DUDA IMPORTANTE:

¿Deberíamos también calcular la moda y promedio de ciertas variables para todos los años disponibles, no solo la última edad?  

Esto nos permitiría comparar cómo ha cambiado el empleado consigo mismo, por ejemplo:

- ¿Se ausentaba más antes o ahora?
- ¿Su hit target mejoró o empeoró con el tiempo?
- ¿Qué tipo de ausencias eran más comunes antes vs. ahora?

### Propuesta 3:
Una posible solución sería crear un segundo DataFrame con "indicadores globales históricos" por empleado, para luego compararlos con su “foto actual”.


# Nuevo DataFrame con una fila por empleado Version 2

In [18]:
# Paso 1: Obtenemos la edad máxima por empleado
edades_max = df_rrhh.groupby('ID')['Age'].max().reset_index()
df_max_age = df_rrhh.merge(edades_max, on=['ID', 'Age']) 

df_max_age



"""Ahora df_max_age tiene solo los registros donde Age es la más alta para ese ID"""

'Ahora df_max_age tiene solo los registros donde Age es la más alta para ese ID'

In [19]:
# Paso 2: Promediar variables fluctuantes por empleado (solo registros de edad máxima)
vars_a_promediar = ['Work_load_Average_day', 'Hit_target', 'Absenteeism_hours']

# (transformo Work_load_Average_day correctamente antes de agrupar. Cambio de comas a puntos y el tipo de V)
df_max_age['Work_load_Average_day'] = (
    df_max_age['Work_load_Average_day']
    .astype(str).str.replace(',', '.', regex=False).astype(float)
)

promedios = df_max_age.groupby('ID')[vars_a_promediar].mean().reset_index()




"""Ahora, work_load_average_day y hit_target representan el promedio durante su edad máxima"""

'Ahora, work_load_average_day y hit_target representan el promedio durante su edad máxima'

In [20]:
# Paso 3: Para datos demográficos y constantes, tomamos una sola fila de perfil general (del registro con edad máxima) 
perfil_estatico = df_max_age.groupby('ID').first().reset_index()

In [26]:
# 1. Total de ausencias históricas (2007–2010)
ausencias_totales = df.groupby('ID').size().reset_index(name='total_ausencias')

# 2. Ausencias en la edad más reciente
# Obtener registros donde cada empleado tenía su edad máxima
edades_max = df.groupby('ID')['Age'].max().reset_index()
df_max_age = df.merge(edades_max, on=['ID', 'Age'])

# Contar cuántas ausencias tienen en esa edad máxima
ausencias_recientes = df_max_age.groupby('ID').size().reset_index(name='ausencias_edad_max')

# 3. Unir ambas al DataFrame de empleados
df_empleados = df_empleados.merge(ausencias_totales, on='ID')
df_empleados = df_empleados.merge(ausencias_recientes, on='ID')



In [ ]:
# Paso 5: Unir todo (perfil + promedios)

df_empleados = perfil_estatico.merge(promedios, on='ID')



"""
Esto Une:
perfil_estatico: una sola fila por empleado con edad máxima y variables constantes
promedios: los valores promedio de variables fluctuantes (como Work_load_Average_day, Hit_target, etc.), pero solo calculados sobre la edad más reciente

"""

In [ ]:
# Paso 6: Limpiar y seleccionar columnas relevantes
columnas_finales = [
    'ID', 'Age', 'Education', 'Son', 'Pet', 'Social_drinker', 'Social_smoker',
    'Weight', 'Height', 'Body_mass_index', 'Transportation_expense',
    'Distance_Residence_Work', 'Service_time', 'Disciplinary_failure',
    'Work_load_Average_day_y', 'Hit_target_y', 'Absenteeism_hours_y',
    'total_ausencias', 'ausencias_edad_max'
]



"""De las variables que fluctúan, incluyo sólo el promedio de la última edad (_y)"""

In [31]:
df_empleados = df_empleados[columnas_finales]

In [32]:
df_empleados

,ID,Age,Education,Son,Pet,Social_drinker,Social_smoker,Weight,Height,Body_mass_index,Transportation_expense,Distance_Residence_Work,Service_time,Disciplinary_failure,Work_load_Average_day_y,Hit_target_y,Absenteeism_hours_y,total_ausencias,ausencias_edad_max
0,1,37,3,1,1,0,0,88,172,29,235,11,14,0,262.894478,95.173913,5.260870,23,23
1,2,48,1,1,5,0,1,88,163,33,235,29,12,0,241.597000,93.333333,4.166667,6,6
2,3,38,1,0,0,1,0,89,170,31,179,51,18,0,262.175062,95.070796,4.265487,113,113
3,4,40,1,1,8,1,0,98,170,34,118,14,13,0,271.219000,95.000000,0.000000,1,1
4,5,43,1,1,0,1,0,106,167,38,235,20,13,0,266.650632,91.526316,5.473684,19,19
5,6,33,1,2,2,0,0,69,167,25,189,29,13,0,274.829000,94.875000,9.000000,8,8
6,7,39,1,2,0,1,1,68,168,24,279,5,14,0,303.210833,94.666667,5.000000,6,6
7,8,39,1,2,2,1,0,100,170,35,231,35,14,1,282.718000,88.000000,0.000000,2,2
8,9,58,1,2,1,0,0,65,172,22,228,14,16,0,249.042250,95.875000,32.750000,8,8
9,10,28,1,1,4,1,0,80,172,27,361,52,3,0,258.870250,94.125000,7.750000,24,24


Ahora faltan las modas...